In [2]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.cluster import KMeans
from sklearn.compose import ColumnTransformer
from sklearn.impute import KNNImputer, SimpleImputer
from sklearn.metrics import accuracy_score, precision_score, recall_score, classification_report, confusion_matrix
from sklearn import ensemble
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from catboost import CatBoostRegressor, Pool
from scipy import stats
import lightgbm as lgb
from scipy.stats import mode
from sklearn.metrics import mean_squared_error, r2_score
import seaborn as sns
from sklearn.feature_selection import SelectKBest, mutual_info_regression

In [3]:
df = pd.read_csv("CreditPrediction.csv")
df

,CLIENTNUM,Customer_Age,Gender,Dependent_count,Education_Level,Marital_Status,Income_Category,Card_Category,Months_on_book,Total_Relationship_Count,Months_Inactive_12_mon,Contacts_Count_12_mon,Credit_Limit,Total_Revolving_Bal,Total_Amt_Chng_Q4_Q1,Total_Trans_Amt,Total_Trans_Ct,Total_Ct_Chng_Q4_Q1,Avg_Utilization_Ratio,Unnamed: 19
0,768805383,45.0,M,3,High School,Married,$60K - $80K,Blue,39.0,5.0,1,3,12691.0,777,1.335,1144,42,1.625,0.061,NaN
1,818770008,49.0,F,5,Graduate,NaN,Less than $40K,Blue,44.0,6.0,1,2,8256.0,864,1.541,1291,33,3.714,0.105,NaN
2,713982108,51.0,M,3,Graduate,Married,$80K - $120K,Blue,36.0,4.0,1,0,3418.0,0,2.594,1887,20,2.333,0.000,NaN
3,769911858,40.0,F,4,High School,NaN,Less than $40K,Blue,34.0,3.0,4,1,3313.0,2517,1.405,1171,20,2.333,0.760,NaN
4,709106358,40.0,M,3,Uneducated,Married,$60K - $80K,NaN,21.0,5.0,1,0,4716.0,0,2.175,816,28,2.500,0.000,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10162,718673358,35.0,M,3,Doctorate,NaN,$80K - $120K,Blue,30.0,5.0,3,4,13590.0,1528,0.728,2137,52,0.486,0.112,NaN
10163,715207458,46.0,F,1,Unknown,Single,Less than $40K,Blue,39.0,2.0,2,1,2029.0,1074,0.514,4802,90,0.800,0.529,NaN
10164,803665983,52.0,M,0,Unknown,NaN,$60K - $80K,Blue,46.0,3.0,2,4,2742.0,2184,0.592,3829,72,0.532,0.796,NaN
10165,713183508,39.0,F,1,High School,NaN,Unknown,NaN,36.0,1.0,3,2,2751.0,1158,0.821,4861,82,0.822,0.421,NaN


In [4]:
z_dataset = pd.DataFrame()
df = df.drop(['CLIENTNUM', 'Unnamed: 19', 'Total_Ct_Chng_Q4_Q1', 'Months_on_book', 'Education_Level', 'Marital_Status', ], axis=1)


z_dataset['Total_Trans_Amt'] = np.abs(stats.zscore(df['Total_Trans_Amt']))
outlAmt = list(np.where(np.abs(z_dataset['Total_Trans_Amt']) > 3)[0])
df = df.drop(df.index[outlAmt])

z_dataset['Total_Trans_Ct'] = np.abs(stats.zscore(df['Total_Trans_Ct']))
outlct = list(np.where(np.abs(z_dataset['Total_Trans_Ct']) > 3)[0])
df = df.drop(df.index[outlct])

missing_values = ["NaN", "Unknown", ""]
df.replace(missing_values, np.nan, inplace=True)

df.info()


<class 'pandas.core.frame.DataFrame'>
Index: 9763 entries, 0 to 10166
Data columns (total 14 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Customer_Age              9763 non-null   float64
 1   Gender                    9568 non-null   object 
 2   Dependent_count           9763 non-null   int64  
 3   Income_Category           8673 non-null   object 
 4   Card_Category             7913 non-null   object 
 5   Total_Relationship_Count  9744 non-null   float64
 6   Months_Inactive_12_mon    9763 non-null   int64  
 7   Contacts_Count_12_mon     9763 non-null   int64  
 8   Credit_Limit              9763 non-null   float64
 9   Total_Revolving_Bal       9763 non-null   int64  
 10  Total_Amt_Chng_Q4_Q1      9763 non-null   float64
 11  Total_Trans_Amt           9763 non-null   int64  
 12  Total_Trans_Ct            9763 non-null   int64  
 13  Avg_Utilization_Ratio     9763 non-null   float64
dtypes: float64(5

In [5]:
categorical_features = df.select_dtypes(include=['object']).columns

df['Income_Category'] = df['Income_Category'].map({"Less than $40K":0, "$40K - $60K":1, "$60K - $80K":2 ,"$80K - $120K":3, "$120K +":4})
df['Card_Category'] = df['Card_Category'].map({"Blue":0, "Silver":1, "Gold":2, "Platinum":3})
# df['Marital_Status'] = df['Marital_Status'].map({"Single":0, "Married":1, "Divorced":2})
# df['Education_Level'] = df['Education_Level'].map({"Uneducated":0, "High School":1, "College":2, "Graduate":3, "Post-Graduate":4, "Doctorate":5})
df['Gender'] = df['Gender'].map({"M":0, "F":1})
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 9763 entries, 0 to 10166
Data columns (total 14 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Customer_Age              9763 non-null   float64
 1   Gender                    9568 non-null   float64
 2   Dependent_count           9763 non-null   int64  
 3   Income_Category           8673 non-null   float64
 4   Card_Category             7913 non-null   float64
 5   Total_Relationship_Count  9744 non-null   float64
 6   Months_Inactive_12_mon    9763 non-null   int64  
 7   Contacts_Count_12_mon     9763 non-null   int64  
 8   Credit_Limit              9763 non-null   float64
 9   Total_Revolving_Bal       9763 non-null   int64  
 10  Total_Amt_Chng_Q4_Q1      9763 non-null   float64
 11  Total_Trans_Amt           9763 non-null   int64  
 12  Total_Trans_Ct            9763 non-null   int64  
 13  Avg_Utilization_Ratio     9763 non-null   float64
dtypes: float64(8

In [9]:
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
from sklearn.impute import SimpleImputer

# مرحله 2: مقداردهی اولیه داده‌های مفقود با میانگین
imputer = SimpleImputer(strategy='mean')
data_imputed = imputer.fit_transform(df)

# تبدیل داده‌های پر شده به DataFrame
df_imputed = pd.DataFrame(data_imputed, columns=df.columns)

# نمایش داده‌های پر شده با میانگین


# مرحله 3: اجرای الگوریتم K-means
kmeans = KMeans(n_clusters=10, random_state=42)
clusters = kmeans.fit_predict(data_imputed)

# اضافه کردن برچسب خوشه به داده‌ها
df_imputed['cluster'] = clusters



# مرحله 4: جایگزینی داده‌های مفقود
for feature in df.columns:
    if df[feature].isnull().any():
        for cluster in range(kmeans.n_clusters):
            # محاسبه میانگین ویژگی برای هر خوشه
            cluster_mean = df_imputed[df_imputed['cluster'] == cluster][feature].mean()
            # جایگزینی داده‌های مفقود با میانگین خوشه
            df_imputed.loc[(df_imputed['cluster'] == cluster) & (df[feature].isnull()), feature] = cluster_mean

# حذف ستون برچسب خوشه
df_final = df_imputed.drop(columns=['cluster'])

# نمایش داده‌های نها

df = df_final


C:\Users\Asus\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\cluster\_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


In [10]:
plt.boxplot(df['Customer_Age'])
plt.title('Customer_Age')
plt.show()

NameError: name 'plt' is not defined

In [ ]:
for column in df.columns:
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    df = df[(df[column] >= lower_bound) & (df[column] <= upper_bound)]

In [ ]:
plt.boxplot(df['Customer_Age'])
plt.title('Customer_Age')
plt.show()

In [ ]:
from scipy import stats

# Box-Cox transformation
box_cox_transformed, _ = stats.boxcox(df['Customer_Age'])
df['Customer_Age'] = box_cox_transformed

box_cox_transformed, _ = stats.boxcox(df['Total_Trans_Amt'])
df['Total_Trans_Amt'] = box_cox_transformed

In [ ]:
features = df.columns.difference(['Credit_Limit'])  # All columns except 'SalePrice'
target = 'Credit_Limit'

# Convert categorical features to strings

categorical_features = df[features].select_dtypes(include=['object']).columns

for feature in categorical_features:

    df[feature] = df[feature].astype(str)
 
# Split data into features and target

X = df[features]
# StandardScaler
# scaler = MinMaxScaler()

# # Fit the scaler to the features and transform them
# X_scaled = scaler.fit_transform(X)


y = df[target]
# Split the data into training and testing sets

# mi = mutual_info_regression(X, y)
# mi_df = pd.DataFrame({'Feature': X.columns, 'Mutual Information': mi})
# mi_df = mi_df.sort_values(by='Mutual Information', ascending=False)
# df = mi_df
# threshold = 0.001  # می‌توانید این آستانه را تغییر دهید
# unimportant_features = mi_df[mi_df['Mutual Information'] < threshold]['Feature'].tolist()

# # حذف ویژگی‌های نامناسب از دیتافریم
# X_selected = X.drop(unimportant_features, axis=1)
# X_selected
# X = X_selected

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
# Find categorical features for CatBoost

categorical_features_indices = np.where(X.dtypes == 'object')[0]


In [ ]:
df

In [ ]:
# from sklearn.feature_selection import SelectKBest, f_regression, RFE
# from sklearn.linear_model import LinearRegression


# selector_filter = SelectKBest(score_func=f_regression, k=5)
# X_train_filtered = selector_filter.fit_transform(X_train, y_train)
# X_test_filtered = selector_filter.transform(X_test)

# # انتخاب ویژگی‌ها بر اساس روش Wrapper (RFE با استفاده از رگرسیون خطی)
# estimator = LinearRegression()
# selector_wrapper = RFE(estimator, n_features_to_select=5, step=1)
# X_train_wrapper = selector_wrapper.fit_transform(X_train, y_train)
# X_test_wrapper = selector_wrapper.transform(X_test)

# # ترکیب فیچرهای انتخاب شده از هر دو روش
# X_train_combined = np.hstack((X_train_filtered, X_train_wrapper))
# X_test_combined = np.hstack((X_test_filtered, X_test_wrapper))

In [ ]:
# Check for missing values

missing_values = df.isnull().sum().sort_values(ascending=False)

missing_values = missing_values[missing_values > 0]

print("\nColumns with missing values:\n", missing_values)

In [ ]:
lgb_reg = lgb.LGBMRegressor()
lgb_reg.fit(X_train, y_train)

y_pred = lgb_reg.predict(X_test)
print(mean_squared_error(y_test, y_pred))

In [ ]:
train_pool = Pool(data=X_train, label=y_train, cat_features=categorical_features_indices)

test_pool = Pool(data=X_test, label=y_test, cat_features=categorical_features_indices)
 
# Train the CatBoost model

model = CatBoostRegressor(
    iterations=10000,
    learning_rate=0.1,
    eval_metric='RMSE',
    logging_level='Verbose',  # Can be 'Silent' to reduce output
    use_best_model=True, nan_mode='Max'
    
)

# Train the model with evaluation
model.fit(
    train_pool,
    eval_set=test_pool,
    early_stopping_rounds=50  # Stops if validation RMSE doesn't improve for 50 rounds
)

In [ ]:
# Make predictions on the test set
from sklearn.metrics import mean_squared_error, r2_score
y_pred = model.predict(test_pool)
# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
 
print(f'Mean squared Error (MSE): {mse:.2f}')
print(f'R2 Score: {r2:.4f}')